# Download the model

In [28]:
from huggingface_hub import hf_hub_download

In [29]:
model_name = 'TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF'

In [30]:
# Define the name of the model file to download.
model_file = 'tinyllama-1.1b-chat-v1.0.Q8_0.gguf'

In [31]:
# Download the model from the Hugging Face Hub and store the 
# path to the downloaded file in the `model_path` variable.
model_path = hf_hub_download(model_name, filename=model_file, local_dir='/home/shiva/DeveloperDen/Python/chat_with_postgres/model')

In [32]:
# Print a message indicating that the model has been downloaded.
print(f'Model downloaded to: {model_path}')

Model downloaded to: /home/shiva/DeveloperDen/Python/chat_with_postgres/model/tinyllama-1.1b-chat-v1.0.Q8_0.gguf


# Initializing the Model

In [33]:
from llama_cpp import Llama

In [34]:
# Initialize a `Llama` object with the downloaded model path.
llm = Llama(
    model_path=model_path,

    # Set the number of context tokens.
    n_ctx=512,

    # Set the number of threads to use.
    n_threads=8,

    # Set the number of GPU layers to work with.
    n_gpu_layers=40
)

# Print a message indicating that the Llama object has been initialized.
print('Llama object initialized successfully.')

llama_model_loader: loaded meta data with 23 key-value pairs and 201 tensors from /home/shiva/DeveloperDen/Python/chat_with_postgres/model/tinyllama-1.1b-chat-v1.0.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = tinyllama_tinyllama-1.1b-chat-v1.0
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          llama.block_count u32              = 22


llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 5632
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 64
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   8:              llama.attention.head_count_kv u32              = 4
llama_model_loader: - kv   9:     llama.attention.layer_norm_rms_epsilon f32              = 0.000010
llama_model_loader: - kv  10:                       llama.rope.freq_base f32              = 10000.000000
llama_model_loader: - kv  11:                          general.file_type u32              = 7
llama_model_loader: - kv  12:                       tokenizer.ggml.model str              = llama
llama_model_loader: - kv  13:                      tokenizer.ggml.tokens arr[str,32000]   = ["<unk>", "<s>", "</s>", "<0x00>", "<...
llama_model_loader: - kv  14:                      tokenizer.ggml.scores arr[f32,32000] 

Llama object initialized successfully.


In [35]:
# Use the Llama object to generate an answer to the question.
output = llm(
    # Prompt
    "<|im_start|>user\nAre you a robot?<|im_end|>\n<|im_start|>assistant\n",

    # Set the maximum number of tokens to generate.
    max_tokens=512,

    # Set the stop sequences to inhttps://www.analyticsvidhya.com/blog/2024/02/sql-generation-in-text2sql-with-tinyllamas-llm-fine-tuning/dicate the end of the generated text.
    stop=["</s>"],
)

# Print the generated text.
print(output['choices'][0]['text'])


llama_print_timings:        load time =    1503.31 ms
llama_print_timings:      sample time =      22.89 ms /   379 runs   (    0.06 ms per token, 16560.34 tokens per second)
llama_print_timings: prompt eval time =    1503.17 ms /    33 tokens (   45.55 ms per token,    21.95 tokens per second)
llama_print_timings:        eval time =   54839.93 ms /   378 runs   (  145.08 ms per token,     6.89 tokens per second)
llama_print_timings:       total time =   57015.75 ms /   411 tokens


In [50]:
def chat_template(question, context):
    """
    Creates a chat template for the Llama model.

    Args:
        question: The question to be answered.
        context: The context information to be used for generating the answer.

    Returns:
        A string containing the chat template.
    """

    template = f"""\
    <|im_start|>user
    Given the database structure, generate an raw SQL query for the following question
    database structure:{context}
    question:{question}
    <|im_end|>
    <|im_start|>assistant 
    """
    # Remove any leading whitespace characters from each line in the template.
    template = "\n".join([line.lstrip() for line in template.splitlines()])
    return template

In [51]:
question = "What is the name of users whose email is shiva.shankar@email.com?"
context = "CREATE TABLE 'users' ('id' VARCHAR NULL DEFAULT NULL,'code' VARCHAR(15) NULL DEFAULT NULL,'first_name' VARCHAR(20) NULL DEFAULT NULL,'middle_name' VARCHAR(20) NULL DEFAULT NULL,'last_name' VARCHAR(20) NULL DEFAULT NULL,'email' VARCHAR(55) NULL DEFAULT NULL,'phone_number' VARCHAR(13) NULL DEFAULT NULL,'date_of_birth' DATE NULL DEFAULT NULL,'department_id' INTEGER NULL DEFAULT NULL,'is_active' BOOLEAN NULL DEFAULT NULL);"
print(chat_template(question,context))

<|im_start|>user
Given the database structure, generate an raw SQL query for the following question
database structure:CREATE TABLE 'users' ('id' VARCHAR NULL DEFAULT NULL,'code' VARCHAR(15) NULL DEFAULT NULL,'first_name' VARCHAR(20) NULL DEFAULT NULL,'middle_name' VARCHAR(20) NULL DEFAULT NULL,'last_name' VARCHAR(20) NULL DEFAULT NULL,'email' VARCHAR(55) NULL DEFAULT NULL,'phone_number' VARCHAR(13) NULL DEFAULT NULL,'date_of_birth' DATE NULL DEFAULT NULL,'department_id' INTEGER NULL DEFAULT NULL,'is_active' BOOLEAN NULL DEFAULT NULL);
question:What is the name of users whose email is shiva.shankar@email.com?
<|im_end|>
<|im_start|>assistant 



In [52]:
# Use the Llama object to generate an answer to the question.
output = llm(
    chat_template(question, context),
    # Set the maximum number of tokens to generate.
    max_tokens=512,
    # Set the stop sequences to indicate the end of the generated text.
    stop=["</s>"],
)

# Print the generated text.
pjrint(output['choices'][0]['text'])

Llama.generate: 13 prefix-match hit, remaining 185 prompt tokens to eval

llama_print_timings:        load time =    1503.31 ms
llama_print_timings:      sample time =       8.17 ms /   144 runs   (    0.06 ms per token, 17621.15 tokens per second)
llama_print_timings: prompt eval time =    6153.96 ms /   185 tokens (   33.26 ms per token,    30.06 tokens per second)
llama_print_timings:        eval time =   18659.85 ms /   143 runs   (  130.49 ms per token,     7.66 tokens per second)
llama_print_timings:       total time =   24958.68 ms /   328 tokens


To answer the question, you can generate the following raw SQL query:
```
SELECT * FROM users WHERE email = 'shiva.sha<|user|>@email.com';
```
In this SQL query, `users` is the name of the table where the email column is located, `id`, `code`, `first_name`, `middle_name`, `last_name`, `email`, `phone_number`, `date_of_birth`, `department_id`, and `is_active` are the column names, and `'<|user|>@email.com'` is the email address of the user whose email you want to retrieve.
